In [54]:
import os.path

import tensorflow as tf
from tensorflow.contrib import slim

import dataset_interval_reader
from shared_examples_queue import SharedExamplesQueue
from models import SequenceAndDnaseClassifier
from trainers import ClassiferTrainer

In [12]:
NUM_TASKS = 6

DATA_DIR = 'test-data'
LOG_DIR = 'test-logs'

INTERVALS_FILE = os.path.join(DATA_DIR, 'intervals_file.json')
INPUTS_FILE = os.path.join(DATA_DIR, 'inputs_file.json')

In [13]:
readers, task_names = dataset_interval_reader.get_readers_and_tasks(INPUTS_FILE, INTERVALS_FILE)
shared_queue = SharedExamplesQueue(readers, batch_size=32)

In [4]:
model = SequenceAndDnaseClassifier(num_tasks=NUM_TASKS)

In [5]:
trainer = ClassiferTrainer(model, epoch_size=5000)

In [6]:
trainer.train(shared_queue, LOG_DIR)

seq_preds shape (32, 1, 928, 25)
dnase_preds shape (32, 1, 928, 25)
concat shape (32, 2, 928, 25)
after combined conv layers shape (32, 1, 904, 55)
INFO:tensorflow:Summary name sequence_conv1/weights:gradient is illegal; using sequence_conv1/weights_gradient instead.
INFO:tensorflow:Summary name sequence_conv1/weights:gradient_norm is illegal; using sequence_conv1/weights_gradient_norm instead.
INFO:tensorflow:Summary name sequence_conv1/biases:gradient is illegal; using sequence_conv1/biases_gradient instead.
INFO:tensorflow:Summary name sequence_conv1/biases:gradient_norm is illegal; using sequence_conv1/biases_gradient_norm instead.
INFO:tensorflow:Summary name sequence_conv2/weights:gradient is illegal; using sequence_conv2/weights_gradient instead.
INFO:tensorflow:Summary name sequence_conv2/weights:gradient_norm is illegal; using sequence_conv2/weights_gradient_norm instead.
INFO:tensorflow:Summary name sequence_conv2/biases:gradient is illegal; using sequence_conv2/biases_gradie

KeyboardInterrupt: 

In [9]:
! ls -lrth {LOG_DIR}

total 9480
-rw-r--r--  1 chris  staff   1.1M Dec  4 20:57 graph.pbtxt
-rw-r--r--  1 chris  staff   1.8K Dec  4 20:57 model.ckpt-0.index
-rw-r--r--  1 chris  staff   1.7M Dec  4 20:57 model.ckpt-0.data-00000-of-00001
-rw-r--r--  1 chris  staff    81B Dec  4 20:57 checkpoint
-rw-r--r--  1 chris  staff   393K Dec  4 20:58 model.ckpt-0.meta
-rw-r--r--  1 chris  staff   278K Dec  4 20:58 tf_trace-1.json
-rw-r--r--  1 chris  staff   1.1M Dec  4 20:59 events.out.tfevents.1480913877.DN51sljf.SUNet


In [14]:
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())
s.run(tf.local_variables_initializer())
_ = tf.train.start_queue_runners(s)

In [55]:
labels = shared_queue.outputs['labels']#.eval()
dataset_idxs = shared_queue.outputs['dataset-index']#.eval()

fc_out = tf.random_normal([32, NUM_TASKS])
logits = fc_out

In [25]:
labels[dataset_idxs == 1,:].eval()

array([ 1, -1, -1, -1,  0,  0])

In [9]:
tf.nn.softmax_cross_entropy_with_logits(fc_out, )

In [19]:
dataset_idxs

array([2, 0, 0, 0, 1, 0, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 2, 1, 2,
       1, 0, 0, 0, 0, 1, 2, 0, 2], dtype=int32)

In [35]:
tf.bool?

In [36]:
tf.boolean_mask?

In [41]:
for dset_idx in range(3):
    print(tf.boolean_mask(labels, tf.equal(dataset_idxs, dset_idx)).eval().shape)

(16, 6)
(12, 6)
(11, 6)


In [44]:
tf.equal(labels, -1)

<tf.Tensor 'Equal_14:0' shape=(32, 6) dtype=bool>

In [46]:
ret = tf.ones_like(labels)
tf.where(tf.equal(labels, -1), tf.zeros_like(labels), tf.ones_like())

TypeError: 'Tensor' object does not support item assignment

In [56]:
sigmoid_xentropy_loss = slim.losses.sigmoid_cross_entropy(logits, labels)

In [57]:
sigmoid_xentropy_loss

<tf.Tensor 'sigmoid_cross_entropy_loss/value:0' shape=() dtype=float32>

In [52]:
labels.dtype

tf.int64

In [53]:
tf.zeros_like?